In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,BatchNormalization

In [ ]:
(x_train,y_train),(x_test,y_test)=keras.datasets.cifar100.load_data()


169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
x_train=x_train/255.0
x_test=x_test/255.0

In [ ]:
model=Sequential()
model.add(keras.Input(shape=(32,32,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(100,activation='softmax'))


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
save_callback=keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/checkpoints/best.keras',
                                              save_weights_only=False,
                                              monitor='val_accuracy',
                                              mode='max',
                                              save_best_only=True)

In [ ]:
reduce_lr=keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)

In [ ]:
model.fit(x_train,y_train,epochs=10,verbose=2,batch_size=64,
          validation_data=(x_test,y_test),callbacks=[save_callback,reduce_lr])

Epoch 1/10
782/782 - 9s - 12ms/step - accuracy: 0.1250 - loss: 3.8097 - val_accuracy: 0.1914 - val_loss: 3.3964 - learning_rate: 1.0000e-03
Epoch 2/10
782/782 - 3s - 4ms/step - accuracy: 0.2373 - loss: 3.1553 - val_accuracy: 0.2703 - val_loss: 3.0536 - learning_rate: 1.0000e-03
Epoch 3/10
782/782 - 3s - 4ms/step - accuracy: 0.2985 - loss: 2.8618 - val_accuracy: 0.3004 - val_loss: 2.8656 - learning_rate: 1.0000e-03
Epoch 4/10
782/782 - 3s - 4ms/step - accuracy: 0.3375 - loss: 2.6629 - val_accuracy: 0.3319 - val_loss: 2.7168 - learning_rate: 1.0000e-03
Epoch 5/10
782/782 - 3s - 4ms/step - accuracy: 0.3710 - loss: 2.4972 - val_accuracy: 0.3433 - val_loss: 2.6469 - learning_rate: 1.0000e-03
Epoch 6/10
782/782 - 5s - 6ms/step - accuracy: 0.3967 - loss: 2.3741 - val_accuracy: 0.3607 - val_loss: 2.5828 - learning_rate: 1.0000e-03
Epoch 7/10
782/782 - 3s - 4ms/step - accuracy: 0.4220 - loss: 2.2643 - val_accuracy: 0.3647 - val_loss: 2.5801 - learning_rate: 1.0000e-03
Epoch 8/10
782/782 - 6s - 

In [ ]:
es_callback=keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=0,
    mode="auto",
    restore_best_weights=False,
    start_from_epoch=0,
)

In [ ]:
model.fit(x_train,y_train,batch_size=64,epochs=10,validation_data=(x_test,y_test),callbacks=[es_callback,save_callback])

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4985 - loss: 1.9161 - val_accuracy: 0.3813 - val_loss: 2.5271
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5034 - loss: 1.8769 - val_accuracy: 0.3850 - val_loss: 2.5361
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5188 - loss: 1.8133 - val_accuracy: 0.3901 - val_loss: 2.4955
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5411 - loss: 1.7400 - val_accuracy: 0.3874 - val_loss: 2.5616
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5427 - loss: 1.7058 - val_accuracy: 0.3922 - val_loss: 2.5141
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5583 - loss: 1.6518 - val_accuracy: 0.3956 - val_loss: 2.5527


In [ ]:
class CustomCallback(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs=None):
    keys=list(logs.keys())
    print(keys,epoch)

In [ ]:
model.fit(x_train,y_train,epochs=2,verbose=2,batch_size=64,
          validation_data=(x_test,y_test),callbacks=[CustomCallback()])

Epoch 1/2
['accuracy', 'loss', 'val_accuracy', 'val_loss'] 0
782/782 - 72s - 92ms/step - accuracy: 0.3982 - loss: 2.3722 - val_accuracy: 0.3631 - val_loss: 2.5794
Epoch 2/2
['accuracy', 'loss', 'val_accuracy', 'val_loss'] 1
782/782 - 76s - 98ms/step - accuracy: 0.4184 - loss: 2.2825 - val_accuracy: 0.3539 - val_loss: 2.6214


In [ ]:
class CustomCallback1(keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs=None):
    if logs.get('accuracy')>0.4:
      print('Quitting Training process')
      self.model.stop_training=True

In [ ]:
model.fit(x_train,y_train,epochs=10,verbose=2,batch_size=64,
          validation_data=(x_test,y_test),callbacks=[CustomCallback1()])

Epoch 1/10
Quitting Training process
782/782 - 79s - 101ms/step - accuracy: 0.4331 - loss: 2.2006 - val_accuracy: 0.3790 - val_loss: 2.5324
